# Create different GWAS enviroments for covariate tests

In [2]:
import pandas as pd
import numpy as np
import os 

In [1]:
# covar constants
INCLUDED_COVARS = ["sex","year_of_birth","AC","batch"]
AC_VALS = ['AC_stockport_pilot','AC_manchester','AC_oxford','AC_cardiff','AC_glasgow','AC_edinburgh','AC_stoke','AC_reading','AC_bury','AC_newcastle','AC_bristol','AC_barts','AC_nottingham','AC_sheffield','AC_liverpool','AC_middlesborough','AC_hounslow','AC_croydon','AC_birmingham','AC_swansea','AC_wrexham','AC_cheadle_revisit','AC_cheadle_imaging','AC_reading_imaging','AC_newcastle_imaging']
BATCH_VALS = ['batch_Batch_b001','batch_Batch_b002','batch_Batch_b003','batch_Batch_b004','batch_Batch_b005','batch_Batch_b006','batch_Batch_b007','batch_Batch_b008','batch_Batch_b009','batch_Batch_b010','batch_Batch_b011','batch_Batch_b012','batch_Batch_b013','batch_Batch_b014','batch_Batch_b015','batch_Batch_b016','batch_Batch_b017','batch_Batch_b018','batch_Batch_b019','batch_Batch_b020','batch_Batch_b021','batch_Batch_b022','batch_Batch_b023','batch_Batch_b024','batch_Batch_b025','batch_Batch_b026','batch_Batch_b027','batch_Batch_b028','batch_Batch_b029','batch_Batch_b030','batch_Batch_b031','batch_Batch_b032','batch_Batch_b033','batch_Batch_b034','batch_Batch_b035','batch_Batch_b036','batch_Batch_b037','batch_Batch_b038','batch_Batch_b039','batch_Batch_b040','batch_Batch_b041','batch_Batch_b042','batch_Batch_b043','batch_Batch_b044','batch_Batch_b045','batch_Batch_b046','batch_Batch_b047','batch_Batch_b048','batch_Batch_b049','batch_Batch_b050','batch_Batch_b051','batch_Batch_b052','batch_Batch_b053','batch_Batch_b054','batch_Batch_b055','batch_Batch_b056','batch_Batch_b057','batch_Batch_b058','batch_Batch_b059','batch_Batch_b060','batch_Batch_b061','batch_Batch_b062','batch_Batch_b063','batch_Batch_b064','batch_Batch_b065','batch_Batch_b066','batch_Batch_b067','batch_Batch_b068','batch_Batch_b069','batch_Batch_b070','batch_Batch_b071','batch_Batch_b072','batch_Batch_b073','batch_Batch_b074','batch_Batch_b075','batch_Batch_b076','batch_Batch_b077','batch_Batch_b078','batch_Batch_b079','batch_Batch_b080','batch_Batch_b081','batch_Batch_b082','batch_Batch_b083','batch_Batch_b084','batch_Batch_b085','batch_Batch_b086','batch_Batch_b087','batch_Batch_b088','batch_Batch_b089','batch_Batch_b090','batch_Batch_b091','batch_Batch_b092','batch_Batch_b093','batch_Batch_b094','batch_Batch_b095','batch_UKBiLEVEAX_b1','batch_UKBiLEVEAX_b10','batch_UKBiLEVEAX_b11','batch_UKBiLEVEAX_b3','batch_UKBiLEVEAX_b4','batch_UKBiLEVEAX_b5','batch_UKBiLEVEAX_b6','batch_UKBiLEVEAX_b7','batch_UKBiLEVEAX_b8','batch_UKBiLEVEAX_b9']
NUMBER_OF_PC = [0,1,5,10,20,40]

# gwas constants 
# PHENOTYPES = ["Multiple_sclerosis","Diastolic_blood_pressure"]
# PHENOTYPES_PAIR = [("Diastolic_blood_pressure","PHENO1.glm.linear"),("Multiple_sclerosis","PHENO1.glm.logistic")]
PHENOTYPES_PAIR = [("Diastolic_blood_pressure","PHENO1.glm.linear"),("Multiple_sclerosis","PHENO1.glm.logistic"),("hypertension",'PHENO1.glm.logistic')]
# PHENOTYPES_PAIR = [("Multiple_sclerosis","PHENO1.glm.logistic")]



PHENOTYPES = list(map(lambda x:x[0],PHENOTYPES_PAIR))
# CHROMOSOMES =  [1]
CHROMOSOMES = list(range(1,23))
RESULT_SUFFIX = "PHENO1.glm.logistic"
NO_COVAR_CHAR = "no_covar"

# scripts constants 
VALUE_AT_END_OF_PHENOTYPES = 'const'
PLINK_PATH = "/cs/usr/nadavb/third_party/plink2"
MEM_USED = 30
MEM_USED_LONG = 30000
CPU_USED = 30
BATCH_LENGTH = 250
MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY = 200
DATA_FILE_NAME = "data_file_2_22_chromosomes.csv"
SAMPLE_FILE = "/cs/biobank-genetics/EGAD00010001474/ukb26664_imp_chr1_v3.sample"
PLINK_GENE_FILES = 0
BGEN_GENE_FILES = 1
HADASSA_PLINK = 2

COMBINE = False

In [3]:
# params
PROJECT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK"
DATASET_FILE = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/ukbb_dataset.csv"
GENE_FILE_TYPE = HADASSA_PLINK

In [4]:
dataset = pd.read_csv(DATASET_FILE)

In [6]:
dataset

,sample_index,eid,Diastolic_blood_pressure,Multiple_sclerosis,hypertension,const,sex,year_of_birth,PC1,PC2,...,batch_UKBiLEVEAX_b1,batch_UKBiLEVEAX_b10,batch_UKBiLEVEAX_b11,batch_UKBiLEVEAX_b3,batch_UKBiLEVEAX_b4,batch_UKBiLEVEAX_b5,batch_UKBiLEVEAX_b6,batch_UKBiLEVEAX_b7,batch_UKBiLEVEAX_b8,batch_UKBiLEVEAX_b9
0,0,5895328,80.0,False,False,1.0,1.0,1942.0,-12.53270,4.235220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5764375,94.0,False,False,1.0,0.0,1951.0,-12.33790,3.061120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5503073,84.0,False,True,1.0,1.0,1944.0,-11.85050,5.194030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2563626,80.0,False,True,1.0,1.0,1950.0,-11.97490,3.477800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4980832,74.0,False,False,1.0,1.0,1948.0,-8.73549,4.980050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333627,502513,2243883,75.0,False,True,1.0,1.0,1949.0,-13.83960,6.899070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
333628,502515,3570729,82.0,False,False,1.0,1.0,1955.0,-15.15060,5.265920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
333629,502516,1663456,79.0,False,False,1.0,1.0,1941.0,-11.59990,2.911470,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
333630,502517,3118989,79.0,False,True,1.0,0.0,1942.0,-13.72620,0.917067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
if not os.path.exists(PROJECT_PATH):
    !mkdir {PROJECT_PATH}

## create covariate files

In [7]:
def create_covar_file(covar_list:list,file_name:str,dataset:pd.DataFrame):
    if os.path.exists(file_name):
        return

    covariate_values = dataset[['eid', 'eid'] + [covariate_col for covariate_col in covar_list if \
            covariate_col != 'const']].dropna()
    covariate_values.to_csv(file_name, header = False, index = False, sep = '\t')


def get_covar_partial_name(covars_to_add,number_of_pcs):
    file_name = str(covars_to_add).replace(", ","_").strip("[]").replace("'","") + f"_{number_of_pcs}_PC"
    if file_name[0] != "_":
        file_name = "_" + file_name
    return file_name

def get_covar_file_name(covars_to_add,number_of_pcs):
    file_name = get_covar_partial_name(covars_to_add,number_of_pcs)
    file_name = os.path.join(PROJECT_PATH, "covariates" + file_name + ".txt")
    return file_name
    
def create_enviroment(number_of_pcs:int,covars_to_add:list,dataset:pd.DataFrame):
    covar_list = []
    for covar in covars_to_add:
        if covar == "AC":
            covar_list.extend(AC_VALS)
        elif covar == "batch":
            covar_list.extend(BATCH_VALS)
        else:
            covar_list.append(covar)
    for i in range(number_of_pcs):
        covar_list.append(f"PC{i+1}")
    file_name = get_covar_file_name(covars_to_add,number_of_pcs)
#     print(file_name)
#     print(str(covars_to_add).replace(", ","_").strip("[]").replace("'","") + f"_{number_of_pcs}_PC_GWAS")
#     !mkdir {dir_name}
    create_covar_file(covar_list,file_name,dataset)
    
def convert_num_to_binary_list(num:int,final_len):
#     return bin(num)
    res = [int(i) for i in list('{0:0b}'.format(num))]
    return [0]*(final_len - len(res)) +res


def get_covars_to_add(val):
    covars_to_add = []
#     create a list of covars for combination (binary list)
    for i,insert in enumerate(convert_num_to_binary_list(val,len(INCLUDED_COVARS))):
        if insert:
            covars_to_add.append(INCLUDED_COVARS[i])
    return covars_to_add

# create all combinations of covars and PC num


In [8]:
df1 = pd.read_csv("/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/covariates_0_PC.txt", sep = '\t') 
df2 = pd.read_csv("/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/covariates_5_PC.txt", sep = '\t') 

In [9]:
for val in range(2**len(INCLUDED_COVARS)):
    print (val,"/",2**len(INCLUDED_COVARS))
    covars_to_add = get_covars_to_add(val)
    for pc_num in NUMBER_OF_PC:
        create_enviroment(pc_num,covars_to_add,dataset)

0 / 16
1 / 16
2 / 16
3 / 16
4 / 16
5 / 16
6 / 16
7 / 16
8 / 16
9 / 16
10 / 16
11 / 16
12 / 16
13 / 16
14 / 16
15 / 16


## create other files

In [8]:
# dataset[["I10"]]
def create_files(gwas_path,dataset):
    !mkdir {gwas_path}/results
    !mkdir {gwas_path}/phenotypes
#     !cp /cs/labs/michall/roeizucker/10krun/runs/0:11/GWAS_delete_me/covariates.txt {gwas_path}/covariates.txt
#     !cd {gwas_path}
#     dataset = pd.read_csv(phen_path)
    for phenotype_col in PHENOTYPES:
        print(phenotype_col)
        file_name = phenotype_col.lower().replace(' ', '_').replace('-', '_') + '.txt'
        values = dataset[['eid', 'eid', phenotype_col]].dropna()
        values[phenotype_col] = values[phenotype_col].astype(float)
        if len(values[phenotype_col].value_counts()) == 2:
            values[phenotype_col] = values[phenotype_col] + 1
        values.to_csv(os.path.join(f"{gwas_path}/phenotypes", file_name), \
            header = False, index = False, sep = '\t')
create_files(PROJECT_PATH,dataset)


mkdir: cannot create directory ‘/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results’: File exists
mkdir: cannot create directory ‘/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/phenotypes’: File exists
Diastolic_blood_pressure
Multiple_sclerosis
hypertension


### create data file

In [5]:
data_file_path = os.path.join(PROJECT_PATH, DATA_FILE_NAME)
mediator_script_path = os.path.join(PROJECT_PATH , "mediator.py")
master_script_path = os.path.join(PROJECT_PATH , "master.sh")


In [9]:
def create_the_data_file(phenotype_cols,gwas_path,data_file_path):
    values = []
    last_counter = 0
    cur_phen_counter = 0
    counter = 0
    for j in CHROMOSOMES:
        base_path =  gwas_path
        if GENE_FILE_TYPE == BGEN_GENE_FILES:
            partial_path = f"/cs/biobank-genetics/EGAD00010001474/ukb_imp_chr{j}_v3.bgen"
        elif GENE_FILE_TYPE == PLINK_GENE_FILES:
            partial_path = f"/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}"
        elif GENE_FILE_TYPE == HADASSA_PLINK:
            partial_path = f"/cs/snapless/michall/hadasak/improve_PRS/bed_files/chr{j}_imputed_snp_id_filtered"
        for phen in phenotype_cols:        
            for val in range(2**len(INCLUDED_COVARS)):
                covars_to_add = get_covars_to_add(val)
                for pc_num in NUMBER_OF_PC:
                    if pc_num == 0 and len(covars_to_add) == 0:
                        continue
                    covar_file_name = get_covar_file_name(covars_to_add,pc_num)           
                    covar_partial_name = get_covar_partial_name(covars_to_add,pc_num)
                    if os.path.exists(os.path.join(base_path ,f"results/{covar_partial_name}_{phen.lower()}_chr{j}.PHENO1.glm.logistic")) or \
                    os.path.exists(os.path.join(base_path ,f"results/{covar_partial_name}_{phen.lower()}_chr{j}.PHENO1.glm.linear")):
                            continue
                    
                    values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                                   os.path.join(base_path, f"results/{covar_partial_name}_{phen.lower()}_chr{j}"),
                                   partial_path,covar_file_name,j])
            counter+=1
            if os.path.exists(os.path.join(base_path ,f"results/{NO_COVAR_CHAR}_{phen.lower()}_chr{j}.PHENO1.glm.logistic"))  or \
            os.path.exists(os.path.join(base_path ,f"results/{covar_partial_name}_{phen.lower()}_chr{j}.PHENO1.glm.linear")):
                continue
            values.append([counter,os.path.join(base_path, "phenotypes/"+ phen.lower() + ".txt"),
                                   os.path.join(base_path, f"results/{NO_COVAR_CHAR}_{phen.lower()}_chr{j}"),
                                   partial_path,NO_COVAR_CHAR,j])
        cur_phen_counter+=1
#         if cur_phen_counter%BATCH_LENGTH == 0:
#             values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
#             last_counter = counter
    df = pd.DataFrame(values, columns=["unsorted_counter","phenotype_path","output_path","partial_chromosome_file_path", "covariates","chr"])
    df.to_csv(data_file_path)
create_the_data_file(PHENOTYPES,PROJECT_PATH,data_file_path)


In [13]:
PROJECT_PATH

'/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK'

In [14]:
df_curr = pd.read_csv(data_file_path)
df_curr
# list(df_curr[df_curr["output_path"].str.contains("_year_of_birth_batch_5_PC_i10_chr4")]["output_path"])

,Unnamed: 0,unsorted_counter,phenotype_path,output_path,partial_chromosome_file_path,covariates,chr


### create scripts

In [10]:
def write_mediator_script(mediator_script_path,data_file_path,gwas_path,master_script_path):
    with open(mediator_script_path, "w") as mediator_file:
        mediator_file.write(f'''import pandas as pd
import sys
import os
import os.path
butch_num = int(sys.argv[1]) 
curr_task = int(sys.argv[2])
BATCH_LENGTH = {BATCH_LENGTH}
SKIP_POINT = {MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY}
PLINK_PATH = "/cs/usr/nadavb/third_party/plink2"

data_file_path = "{data_file_path}"
df = pd.read_csv(data_file_path)
print(df.loc[butch_num * BATCH_LENGTH + curr_task])
val = df.loc[butch_num * BATCH_LENGTH + curr_task]
location = butch_num * BATCH_LENGTH + curr_task
SAMPLE_FILE = "/cs/biobank-genetics/EGAD00010001474/ukb26664_imp_chr" + str(df.loc[location].chr) + "_v3.sample"

if {GENE_FILE_TYPE == BGEN_GENE_FILES}:
    command_base = PLINK_PATH + " --bgen " + df.loc[location].partial_chromosome_file_path + " --sample " + SAMPLE_FILE
elif {GENE_FILE_TYPE == PLINK_GENE_FILES}:
    command_base = PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam"
elif {GENE_FILE_TYPE == HADASSA_PLINK}:
    command_base = PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam"
if curr_task == SKIP_POINT and location < 30500:
    new_batch  = butch_num + 1
    if not os.path.isfile(os.path.join("{gwas_path}", "_" + str(new_batch) + "_flag")):
        os.system("sbatch --array=0-"+str(BATCH_LENGTH - 1)+" --mem={MEM_USED + 2}g -c{CPU_USED} --time=3-0 --killable --requeue --wrap=\\"{master_script_path} "+str(new_batch)+"\\"")
if location < 30500:
    if df.loc[location].covariates != "{NO_COVAR_CHAR}":
        print(command_base +" --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads {CPU_USED} --memory {MEM_USED_LONG}",flush=True)
        os.system(command_base +" --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads {CPU_USED} --memory {MEM_USED_LONG}")
        print("done!")
    else:
        print(command_base +" --pheno " + df.loc[location].phenotype_path + " --out  " + df.loc[location].output_path + " --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads {CPU_USED} --memory {MEM_USED_LONG}",flush=True)
        os.system(command_base +" --pheno " + df.loc[location].phenotype_path + " --out  " + df.loc[location].output_path + " --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads {CPU_USED} --memory {MEM_USED_LONG}")
        print("done!")
''')

def write_master_script(master_script_path,mediator_script_path,gwas_path):
    with open(master_script_path, "w") as master_file:
        master_file.write(f'''
f1="{os.path.join(gwas_path,"_$1_flag")}"
if [ ! -f "$f1"  ]
then 
    touch {os.path.join(gwas_path,"_$1_flag")}
fi
python {mediator_script_path} $1 $SLURM_ARRAY_TASK_ID
''')
    !chmod 744 {master_script_path}
write_mediator_script(mediator_script_path,data_file_path,PROJECT_PATH,master_script_path)
write_master_script(master_script_path,mediator_script_path,PROJECT_PATH)
print(f'''how to run:
sbatch --array=0-263 --mem={MEM_USED + 2}g -c{CPU_USED} --time=3-0 --requeue --killable --wrap="{master_script_path} 0"''')

how to run:
sbatch --array=0-263 --mem=32g -c30 --time=3-0 --requeue --killable --wrap="/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/master.sh 0"


In [16]:

sbatch --array=0-1 --mem=32g -c30 --time=3-0 --requeue --killable --wrap="/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/master.sh 0"

!cat {mediator_script_path}

SyntaxError: invalid syntax (1470108841.py, line 1)

In [ ]:
# # tests
# df = pd.read_csv("/sci/nosnap/michall/roeizucker/covar_tests/results/_0_PC_i10_chr1.PHENO1.glm.logistic",sep="\t")
# df1 = pd.read_csv("/sci/nosnap/michall/roeizucker/covar_tests/results/_40_PC_i10_chr1.PHENO1.glm.logistic",sep="\t")
# # df = df[df["P"] > 0]
# # df1 = df1[df1["P"] > 0]
# # df["P"].idxmin()
# print(df.loc[[df["P"].idxmin()]])
# print(df1.loc[[df1["P"].idxmin()]])
if not COMBINE:
    input()

## combine results

In [11]:
for phen_pair in PHENOTYPES_PAIR:
    if phen_pair[0] != "Multiple_sclerosis":
        continue
    failed = False
    dfs = []
    for j in CHROMOSOMES:    
        if j == 2:
            continue
#     for j in CHROMOSOMES:    
        for val in range(2**len(INCLUDED_COVARS)):
            covars_to_add = get_covars_to_add(val)
            for pc_num in NUMBER_OF_PC:
                if pc_num == 0 and len(covars_to_add) == 0:
                    continue
                covar_file_name = get_covar_file_name(covars_to_add,pc_num)           
                covar_partial_name = get_covar_partial_name(covars_to_add,pc_num)
                result_file_path = os.path.join(PROJECT_PATH, f"results/{covar_partial_name}_{phen_pair[0].lower()}_chr{j}.{phen_pair[1]}")
                if not os.path.exists(result_file_path):
                    print("file does not exist:",result_file_path)
                    failed = True
                    continue
                try:
                    df = pd.read_csv(os.path.join(PROJECT_PATH, f"results/{covar_partial_name}_{phen_pair[0].lower()}_chr{j}.{phen_pair[1]}"),sep="\t")
                except:
                    failed = True
                    print("error for",result_file_path)
                    continue
                for covar in INCLUDED_COVARS:
                    df[covar] = 0
                for covar in covars_to_add:
                    df[covar] = 1
                df["covars"] = ("_".join(covars_to_add))
                df["pc_num"] = pc_num
                dfs.append(df)
            covar_file_name = get_covar_file_name(covars_to_add,pc_num)
        df = pd.read_csv(os.path.join(PROJECT_PATH, f"results/{NO_COVAR_CHAR}_{phen_pair[0].lower()}_chr{j}.{phen_pair[1]}"),sep="\t")
        df["covars"] = ""
        df["pc_num"] = 0
        dfs.append(df)
    if not failed:
        master_df = pd.concat(dfs)
        master_df.to_csv(os.path.join(PROJECT_PATH,f"results_{phen_pair[0]}.csv"),index=False)
        print(os.path.join(PROJECT_PATH,f"results_{phen_pair[0]}.csv"))

/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results_Multiple_sclerosis.csv


In [ ]:
# df = pd.read_csv(data_file_path)
# # list(df["output_path"][:10])
# df[df["output_path"] == "/sci/nosnap/michall/roeizucker/covar_tests/results/_AC_batch_40_PC_i10_chr6"]["covariates"][0]
# # df = pd.read_csv("/sci/nosnap/michall/roeizucker/covar_tests/results/_AC_batch_40_PC_i10_chr6.PHENO1.glm.logistic",sep="\t")

In [ ]:
val = os.path.join(PROJECT_PATH,f"results_{phen}.csv")
!head {val}

In [ ]:
master_df = pd.concat(dfs)


In [ ]:


master_df.to_csv(os.path.join(PROJECT_PATH,"results.csv"),index=False)

In [ ]:
output_folder_path = os.path.join(PROJECT_PATH,"output")
for filename in os.scandir(output_folder_path):
    if filename.is_file():
        with open(filename.path,"r") as file:
            file_content = file.read()
            if "done!" not in file_content and "covar_testsmaster" not in file_content and "print(df.loc[butch_num * BATCH_LENGTH + curr_task])" not in file_content:
                print(filename)


In [ ]:
master_df["#CHROM"].value_counts()